# [Langsmith](https://python.langchain.com/v0.1/docs/langsmith/walkthrough/)

In [66]:
%pip install -U langgraph langsmith
%pip install langchain-openai
%pip install python-dotenv
%pip install --upgrade --quiet  langchain langchainhub
%pip install --upgrade --quiet  tiktoken pandas duckduckgo-search
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [67]:
from dotenv import load_dotenv, find_dotenv
import os
dotenv = find_dotenv()
print(f"Found .env file at {dotenv}")
load_dotenv(dotenv_path=dotenv, override=True)

from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

Found .env file at /home/guilherme-alves-carvalho/Documents/repos/dsc-langraph/.env


In [68]:
from typing import Annotated, Literal, TypedDict

from langchain_core.tracers.context import tracing_v2_enabled
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI



from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

import os
import json

from langchain import hub
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI
from langsmith import Client
from openai import ChatCompletion

CORTEX_ENDPOINT = os.environ.get("CORTEX_ENDPOINT")
MERCURY_ENDPOINT = os.environ.get("MERCURY_ENDPOINT")

# https://smith.langchain.com/hub/wfh/langsmith-agent-prompt?organizationId=3e9f24f6-5a49-5652-a085-d776844de3bf
# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

# Define the tools for the agent to use
tools = [
    DuckDuckGoSearchResults(
        name="duck_duck_go"
    ),  # General internet search using DuckDuckGo
]


tool_node = ToolNode(tools)

# fork for request_group (init/end chat) add tags, fields (TODO)
model = ChatOpenAI(
    model="7b-mistral-balanced_loan_math_tools-loan_no_tools-rank32-scaling2-dropout01-lora_all",
    temperature=0,
    base_url=CORTEX_ENDPOINT,
)


class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        # Customize the JSON serialization for non-serializable objects
        try:
            return super().default(obj)
        except TypeError:
            return str(obj)

class Wrapper:
    def __init__(self, wrapped_class):
        self.wrapped_class = wrapped_class

    def __getattr__(self, attr):
        original_func = getattr(self.wrapped_class, attr)

        def wrapper(*args, **kwargs):
            kwargs_json = json.dumps(kwargs, indent=4, cls=CustomJSONEncoder)
            print(f"Request: {kwargs_json}")
            result = original_func(*args, **kwargs)
            data = json.loads(result.model_dump_json())
            json_str = json.dumps(data, indent=4, cls=CustomJSONEncoder)
            print(f"Response: {json_str}")
            return result

        return wrapper

model.client = Wrapper(model.client)

model = model.bind_tools(tools)

class State(MessagesState):
    metadata: Annotated[dict, "The configuration for the workflow"]

# Define the function that determines whether to continue or not
def should_continue(state: State) -> Literal["tools", "agent", END]:
    messages = state['messages']
    last_message = messages[-1]

    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"

    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: State):
    messages = state['messages']
    response = model.bind(extra_body=state['metadata']).invoke(messages)
    # We return a list, because this will get added to the existing list

    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

{
            "content": "[snippet: Uberlndia Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Uberlndia area., title: Uberl\u00e2ndia, Brazil Weather Conditions | Weather Underground, link: https://www.wunderground.com/weather/br/uberl\u00e2ndia], [snippet: 12 day Uberl\u00e2ndia Weather Forecast. Live Weather Warnings, hourly weather updates. Accurate Uberl\u00e2ndia weather today, forecast for sun, rain, wind and temperature., title: Uberl\u00e2ndia Weather Forecast, link: https://www.weather-forecast.com/locations/Uberlandia/forecasts/latest], [snippet: Current weather in Uberl\u00e2ndia and forecast for today, tomorrow, and next 14 days, title: Weather for Uberl\u00e2ndia, Minas Gerais, Brazil - timeanddate.com, link: https://www.timeanddate.com/weather/@6321899], [snippet: In Uberl\u00e2ndia, the wet season is humid and overcast, the dry season is mostly clear, and it is warm year round. Over the course of the year, the temperature typically varies from 58\u00b0F to 85\u00b0F and is rarely below 53\u00b0F or above 92\u00b0F., title: Uberl\u00e2ndia Climate, Weather By Month, Average Temperature (Minas Gerais ..., link: https://weatherspark.com/y/30110/Average-Weather-in-Uberl\u00c3\u0192\u00c2\u00a2ndia-Brazil-Year-Round]",
            "role": "assistant",
        }

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable

langsmith_client = Client(api_url=MERCURY_ENDPOINT)
# Neosops client (fork langsmith_client) (TODO)
with tracing_v2_enabled(client=langsmith_client):
    # INFERENCE REQUEST DATA 
    # customer_id
    # group_id

    question = HumanMessage(content=str(input()))
    state = {"messages": [SystemMessage(content="Você é um agente especialista em clima, você é a mulher do tempo do jornal nacional"), question], "metadata": {"user_id": "123"}}
     


    # "Ola, qual a temperatura de uberlândia?"
    while question != "q": 
        print(state)
        final_state = app.invoke(
            state,
            config={"thread_id": 42, "workflow": workflow},
        )
        question = HumanMessage(content=str(input()))
    # app.invoke(final_state, config={"thread_id": 42, "workflow": workflow})
    # print(final_state["messages"][-1].content)

{'messages': [SystemMessage(content='Você é um agente especialista em clima, você é a mulher do tempo do jornal nacional'), HumanMessage(content='Ola, gostaria de saber a temperatura de uberlândia')], 'metadata': {'user_id': '123'}}
Request: {
    "messages": [
        {
            "content": "Voc\u00ea \u00e9 um agente especialista em clima, voc\u00ea \u00e9 a mulher do tempo do jornal nacional",
            "role": "system"
        },
        {
            "content": "Ola, gostaria de saber a temperatura de uberl\u00e2ndia",
            "role": "user"
        }
    ],
    "model": "7b-mistral-balanced_loan_math_tools-loan_no_tools-rank32-scaling2-dropout01-lora_all",
    "stream": false,
    "n": 1,
    "temperature": 0.0,
    "logprobs": false,
    "tools": [
        {
            "type": "function",
            "function": {
                "name": "duck_duck_go",
                "description": "A wrapper around Duck Duck Go Search. Useful for when you need to answer questions abo

InternalServerError: Error code: 500 - {'error': 'error, status code: 400, message: %!s(<nil>)'}